Import du contenu des lettres dans une table de travail a partir
de laquelle va se faire le nettoyage des données
Les lettres du pere noel, dezippee, sont dans le dossier < /Partie_1/lettres_pere_noel >

In [ ]:
import psycopg2
import os
import csv
import pandas as pd

In [ ]:
def chemin_fichier()-> str :
    """
        Donne le nom du chemin complet du fichier actuel (.py) 
    """
    chemin=os.getcwd() #methode pour tourner avec jupyter
    #chemin= path.dirname(path.abspath(__file__)) #methode d'Alice avec <os>
    #chemin = str(Path(__file__).parent) #ma methode avec <path>
    #print(chemin)
    return chemin


In [ ]:
# etablir une connection
def connexion_DB(fichierDB):
    try:
        conn = psycopg2.connect(user = "santa",
                                password = "claus",
                                host = "localhost",
                                port = "5432:5432",
                                database = fichierDB
                                )

        cur = conn.cursor()
        # Print PostgreSQL Connection properties
        #print ( "Proprietes : ",conn.get_dsn_parameters(),"\n")

        # Print PostgreSQL version
        cur.execute("SELECT version();")
        record = cur.fetchone()
        #print("Version de PostgreSQL : ", record,"\n")
        cur.close()
    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)
    
    return conn

In [ ]:
def deconnexion_DB(conn):
#closing database connection.
    if(conn):
        conn.close()
        print("PostgreSQL connection is closed")

In [ ]:
conn=connexion_DB("santa_data")

path_dossier_letters=chemin_fichier() +'/Partie_1/lettres_pere_noel'

In [ ]:
# lecture de chaque lettre et import dans la table de travail T_complete
try:
    conn=connexion_DB("santa_data")
    cursor = conn.cursor()
    
    #suppression des enregistrements de la T_complete avant de refaire tourne le programme
    cursor.execute("DELETE FROM T_complete")
    
    #lecture de la T_letters et extraction complete de (Id_l et nom_fichier)
    cursor.execute("SELECT * FROM T_lettres")
    resultat=cursor.fetchall() #res : liste de tuples
    
    count=0
    for tple in resultat:
        #print(tple[0],' et ',tple[1]) 
        #lecture du fichier lettre  correspondant a tpl(1)<lettre_....pnoel>''
        path_lettre=path_dossier_letters +'/'+ tple[1]
        with open(path_lettre, "r", encoding="WINDOWS-1258") as data:
            data = data.read()
            #construction du tuple des valeurs a inserrer dans la table T_complete
            d_insert=[]
            d_insert.append(tple[0]) # l'Id_lettre
            d_insert.append(tple[1]) # le nom fe fichier
            #print(d_insert)
            df_l=pd.DataFrame(data.splitlines()) # df en 1 colonne de 5 lignes
            df_l=df_l.T # transpose en 1 ligne de 5 colonnes
            
            for colonne in df_l:
                d_insert.append(df_l[colonne][0].lstrip()) #ajouter la valeur de la colonne dans la data_values
                #print(d_insert)
            data_values=tuple(d_insert)
            #print('data_values : ',data_values)
    
            insert_data_query = f""" 
                                INSERT INTO T_complete
                                (ID,id_l,nom_lettre,nom_enfant,age,Pays,Contenu,Numero) 
                                VALUES (default,%s,%s,%s,%s,%s,%s,%s)
                                """      
            cursor.execute(insert_data_query,data_values)
            conn.commit()
            count += cursor.rowcount
    print (count, "Record inserted successfully into T_Complete")

except (Exception, psycopg2.Error) as error :
    if(conn):
        print("Failed to insert record into table T_complete", error)

finally:
    #closing database connection.
    if(conn):
        cursor.close()
        deconnexion_DB(conn)

In [ ]:
# rcontrole du resultat sur la table T_complete 
conn=connexion_DB("santa_data")
cursor = conn.cursor()

cursor.execute("SELECT Pays,nom_enfant,age,contenu FROM T_complete order by Age;")
    #resultats=cur.execute(select_SQL)'
res=cursor.fetchall()
#print(res)

print('\nPays'.ljust(15),'nom_enfant'.ljust(25),'age'.ljust(30),
    'Contenu'.ljust(10),''.ljust(15))
trait="-"*90
print(trait)
for row in res:
    print(str(row[0]).ljust(15),end='')
    print(str(row[1]).ljust(25),end='')
    print(str(row[2]).ljust(30),end='')
    print(str(row[3])[0:30].ljust(10))
    

cursor.close()
deconnexion_DB(conn)
